<a href="https://colab.research.google.com/github/thiagokrugel/tfidf-cosseno/blob/main/tfidf_cosseno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Feito por: Thiago Krügel
'''
ENUNCIADO 
Sua tarefa será gerar a matriz termo-documento usando TF-IDF por meio da aplicação das 
fórmulas  TF-IDF  na  matriz  termo-documento  criada  com  a  utilização  do  algoritmo  Bag of 
Words. Sobre o Corpus que recuperamos anteriormente. O entregável desta tarefa é uma 
matriz termo-documento onde a primeira linha são os termos e as linhas subsequentes são 
os vetores calculados com o TF-IDF. 
2. Sua tarefa será gerar uma matriz de distância, computando o cosseno do ângulo entre todos 
os vetores que encontramos usando o tf-idf. Para isso use a seguinte fórmula para o cálculo 
do  cosseno  use  a  fórmula  apresentada  em  Word2Vector  (frankalcantara.com) 
(https://frankalcantara.com/Aulas/Nlp/out/Aula4.html#/0/4/2)  e  apresentada  na  figura  a 
seguir:  
(figura)
O resultado deste trabalho será uma matriz que relaciona cada um dos vetores já calculados 
com todos os outros vetores disponíveis na matriz termo-documento mostrando a distância 
entre cada um destes vetores. 
'''
from bs4 import BeautifulSoup
import requests
import spacy
import string
import numpy as np

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe('sentencizer')

sites = [
  'https://hbr.org/2022/04/the-power-of-natural-language-processing',
  'https://en.wikipedia.org/wiki/Natural_language_processing', 
  'https://www.datarobot.com/blog/what-is-natural-language-processing-introduction-to-nlp/', 
  'https://www.sas.com/en_us/insights/analytics/what-is-natural-language-processing-nlp.html',
  'https://monkeylearn.com/natural-language-processing/'
]
corpuses = []

for site in sites:
  pagina = requests.get(site)
  if pagina.status_code != 200:
    print(f"Ocorreu um erro ao buscar a página {site}.");
    continue
  soup = BeautifulSoup(pagina.content, 'html.parser') 
  corpus = []
  for tagParagrafo in soup.find_all(["p", "h1", "h2", "h3", "h4", "h5", "h6", "li" ]): 
    paragrafo = tagParagrafo.get_text()
    strippedParagrafo = paragrafo.strip()
    if len(strippedParagrafo) > 0:
      doc = nlp(strippedParagrafo)
      for sentença in doc.sents:
        corpus.append(sentença.text.translate(str.maketrans('', '', string.punctuation)))
  corpuses.append(corpus)


lexemas = []
numeroSentencas = 0
for corpus in corpuses:
  for sentencas in corpus:
    numeroSentencas += 1
    for lexema in sentencas.split(' '):
      if lexema not in lexemas:
        lexemas.append(lexema)
bagOfWords = np.zeros((numeroSentencas,len(lexemas)))
sentencaAtual = 0
for corpus in corpuses:
  for sentencas in corpus:
    for lexema in sentencas.split(' '):
      bagOfWords[sentencaAtual][lexemas.index(lexema)] += 1
    sentencaAtual += 1


tfs = np.zeros((len(bagOfWords),len(bagOfWords[0])))
sentencaAtual = 0
for corpus in corpuses:
  for sentencas in corpus:
    numeroDeLexemas = len(sentencas.split(' '))
    for lexema in sentencas.split(' '):
      tfs[sentencaAtual][lexemas.index(lexema)] = bagOfWords[sentencaAtual][lexemas.index(lexema)] / numeroDeLexemas
    sentencaAtual += 1
print("TF: ")
print(tfs)

idfs = []
for lexema in range(len(lexemas)):
  lexemeRate = 0
  for sentenceRates in bagOfWords:
    if sentenceRates[lexema] > 0: lexemeRate += 1
  idfs.append(np.log10(len(bagOfWords)/lexemeRate))
print("IDF: ")
print(len(bagOfWords))
print(idfs)

tfidf = np.zeros((len(bagOfWords),len(bagOfWords[0])))
for i in range(len(tfidf)):
  for j in range(len(tfidf[0])):
    tfidf[i][j] = tfs[i][j] * idfs[j]
print('TF-IDF: ')
print(tfidf)



TF: 
[[0.5 0.5 0.  ... 0.  0.  0. ]
 [0.  0.  1.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 ...
 [0.  0.  0.  ... 0.  0.  1. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]]
IDF: 
2076
[3.31722734917642, 3.31722734917642, 3.31722734917642, 2.840106094456758, 1.5390760987927767, 2.840106094456758, 3.016197353512439, 2.7151673578484576, 2.840106094456758, 3.016197353512439, 2.414137362184477, 3.016197353512439, 2.3629848397370954, 2.7151673578484576, 3.016197353512439, 3.31722734917642, 2.4721293091621632, 3.31722734917642, 3.31722734917642, 3.31722734917642, 3.016197353512439, 3.31722734917642, 3.31722734917642, 1.9554995131588273, 2.840106094456758, 2.7151673578484576, 3.016197353512439, 2.061954844073114, 1.1499100144282441, 3.016197353512439, 2.840106094456758, 3.016197353512439, 3.016197353512439, 3.016197353512439, 2.2380461031287955, 2.1710993134981824, 2.7151673578484576, 2.061954844073114, 2.5390760987927767, 2.840106094456758, 3.016197353512439, 3.0161973

In [12]:
currentVetor = 0
matrizDeDistancia = np.zeros((len(tfidf), len(tfidf)))

for vetor in tfidf:
  i = currentVetor
  while i < len(tfidf):
    distance = np.dot(vetor, tfidf[i])/(np.linalg.norm(vetor)*np.linalg.norm(tfidf[i]))
    matrizDeDistancia[currentVetor][i] = distance
    matrizDeDistancia[i][currentVetor] = distance
    i += 1
  currentVetor += 1

print(matrizDeDistancia)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
